In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from ipywidgets import interactive, interact, IntSlider

## Choose train/test/val and SV type

In [33]:
def get_data_split(train_test='train'): return train_test
w1 = interactive(get_data_split, train_test=['train', 'test', 'val'])
display(w1)

interactive(children=(Dropdown(description='train_test', options=('train', 'test', 'val'), value='train'), Out…

In [3]:
def get_svtype(svtype='nosv'): return svtype
w2 = interactive(get_svtype, svtype=['nosv', 'del', 'dup'])
display(w2)

interactive(children=(Dropdown(description='svtype', options=('nosv', 'del', 'dup'), value='nosv'), Output()),…

## Load Bed files

In [46]:
TRAIN_TEST = w1.result
SVTYPE = w2.result

df = pd.read_csv(f'../data/pileups/{TRAIN_TEST}/{TRAIN_TEST}_{SVTYPE}.bed',
                 sep='\t', 
                 names=['chrom', 'start', 'end', 'data'] if SVTYPE == 'nosv'
                       else ['chrom', 'start', 'end', 'genotype', 'data']
                )
df.dropna(axis=0, inplace=True)
print(len(df))
df.head()

398


,chrom,start,end,genotype,data
0,21,14982500,14983000,0|1,"2,0,0,0,1,0,0,0,0,0,0,4,0,0,0,1,5,0,0,0,3,0,0,..."
1,21,14983000,14983120,0|1,"2,0,0,0,1,0,0,0,2,0,0,0,1,0,0,0,0,0,0,1,0,0,0,..."
2,21,14987668,14987941,0|1,"6,0,0,0,4,0,0,0,0,3,0,0,1,2,0,0,0,2,0,0,0,2,0,..."
3,21,14990620,14990826,0|1,"2,0,0,0,1,0,0,0,0,0,3,0,0,0,1,0,3,0,0,0,1,0,0,..."
4,21,14992429,14992666,0|1,"3,0,0,0,2,0,0,0,3,0,0,0,1,0,0,0,0,3,0,0,0,1,0,..."


### Convert data field into matrix of size (8, N)

In [47]:
df.data = df.data.apply(lambda x: np.fromstring(x, sep=',').reshape((-1, 8)).T)

### Visualize

In [48]:
@interact(i=IntSlider(min=0, max=len(df)-1, step=1))
def plot_example(i=0) :
    matplotlib.rc('font', **dict(size=100))
    plt.figure(figsize=(100, 100))
    sns.heatmap(df.data.values[i], xticklabels=False, yticklabels=False,
                square=True, cbar_kws={"orientation": "horizontal"})

interactive(children=(IntSlider(value=0, description='i', max=397), Output()), _dom_classes=('widget-interact'…

### Write Data

In [45]:
pd.to_pickle(df, f'../data/DataFrames/{TRAIN_TEST}/{TRAIN_TEST}_{SVTYPE}.pkl')